In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Name Scopes

W przypadku bardziej złożonych modeli, takich jak sieci neuronowe, wykres może być skomplikowany (mamy tysiące węzłów). Aby tego uniknąć, możesz utworzyć zakresy nazw, aby grupować powiązane węzły. Na przykład zmodyfikujmy poprzedni kod, aby zdefiniować **error** i **mse** w name scopes **loss**:

```python
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
```

# Zad.
Wczytaj dane.

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# Zawsze unormuj dane

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
housing_data_scaled=scaler.fit_transform(housing.data)

In [4]:
X_train =  housing_data_scaled
y_train = housing.target.reshape(-1, 1)

# Zad.
  * Proszę narysować wykres zmiany funkcji kosztu w przypadku:
    * Gradient Descent
    * Momentum
  dla 100 iteracji.

  * Następnie zapisać model.

  * I wykonać następne 100 iteracji po ponownym wczytaniu.
  

In [5]:
data_plus_bias  = np.c_[np.ones((X_train.shape[0], 1)), X_train]
data_m, data_n = data_plus_bias.shape
print(data_m, data_n)

20640 9


In [6]:
logs_path = "./tf_logs/run_number_1_solution/"

tf.reset_default_graph()

n_epochs = 100
batch_size = 64
learning_rate = 0.01

error_array_g_b=[]
error_array_m_b=[]

batch_error_array_g=[]
batch_error_array_m=[]


with tf.name_scope("reader"):
    X = tf.placeholder(tf.float32, shape=(None, data_n), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    theta = tf.Variable(tf.random_uniform([data_n, 1], -1.0, 1.0, seed=42), name="theta")
    y_pred = tf.matmul(X, theta, name="predictions")
    
with tf.name_scope("loss") as scope:
    
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

with tf.name_scope("train"):    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)

    init = tf.global_variables_initializer()


# Create a summary to monitor cost tensor
tf.summary.scalar("mse", mse)
# Create a summary to monitor accuracy tensor
merged_summary_op = tf.summary.merge_all()


n_batches = int(np.ceil(data_m / batch_size))

def fetch_batch(epoch, batch_index, batch_size, m):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = data_plus_bias[indices] 
    y_batch = y_train[indices] 
    return X_batch, y_batch

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    ###############################    
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    ###############################
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size, data_m)
            #sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            ############################### 
            _, summary = sess.run( [training_op, merged_summary_op] , feed_dict={X: X_batch, y: y_batch})
            summary_writer.add_summary(summary, epoch * n_batches + batch_index)
            ###############################
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE =", sess.run( mse, feed_dict={X: X_batch, y: y_batch}) )   
        
        batch_error_array_g.append( sess.run( mse, feed_dict={X: X_batch, y: y_batch}) )
        error_array_g_b.append( sess.run( mse, feed_dict={X: data_plus_bias, y: y_train}) )            
    
    best_theta_g = theta.eval()
    save_path = saver.save(sess, "./regression/my_model_final_g.ckpt")
    
print("Best theta:")
print(best_theta_g)

Epoch 0 MSE = 0.31841922
Epoch 10 MSE = 0.8001851
Epoch 20 MSE = 0.5560062
Epoch 30 MSE = 0.44753096
Epoch 40 MSE = 0.54900646
Epoch 50 MSE = 0.80266595
Epoch 60 MSE = 0.55392677
Epoch 70 MSE = 0.43071902
Epoch 80 MSE = 0.27837253
Epoch 90 MSE = 0.7054215
Best theta:
[[ 2.0728784 ]
 [ 0.8359189 ]
 [ 0.11120823]
 [-0.25528795]
 [ 0.29422897]
 [-0.00615538]
 [-0.04408351]
 [-0.9086386 ]
 [-0.8651398 ]]


# Uruchom w konsoli
```python
tensorboard --logdir ./tf_logs

tensorboard --logdir ./tf_logs
```